In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import pymysql
import os # Untuk membuat folder

print("Library berhasil diimpor!")

Library berhasil diimpor!


# Mengambil Database

In [3]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',                # Ganti dengan username MySQL Anda
    'password': '', # Ganti dengan password MySQL Anda
    'database': 'smart_clothesline_db'
}

query = """
SELECT 
    temperature, 
    humidity, 
    rain_value, 
    ldr_value, 
    status_jemuran 
FROM 
    jemuran_data
WHERE 
    status_jemuran IS NOT NULL AND status_jemuran != '' -- Pastikan hanya mengambil data dengan status_jemuran yang valid
ORDER BY waktu ASC 
"""

try:
    conn = pymysql.connect(**DB_CONFIG)
    df = pd.read_sql(query, conn)
    conn.close()
    print("Data berhasil diambil dari database MySQL!")
    print(f"Jumlah baris data: {len(df)}")
    if len(df) == 0:
        print("PERINGATAN: Database kosong atau query tidak mengembalikan data yang valid.")
        print("Pastikan Anda memiliki data di tabel 'rain_data' dan kolom 'status_jemuran' terisi.")
        # Hentikan eksekusi jika tidak ada data untuk menghindari error selanjutnya
        raise ValueError("Tidak ada data yang cukup untuk melatih model.")

except Exception as e:
    print(f"ERROR: Gagal mengambil data dari MySQL: {e}")
    print("Pastikan server MySQL berjalan, kredensial benar, dan tabel/kolom ada.")
    print("Pastikan juga kolom 'status_jemuran' memiliki nilai (tidak NULL/kosong).")
    raise # Rerise exception untuk menghentikan notebook jika ada error kritis

Data berhasil diambil dari database MySQL!
Jumlah baris data: 149


C:\Users\USER\AppData\Local\Temp\ipykernel_9688\1231017429.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


# Pra-pemrosesan Data

In [5]:
print("\n5 baris pertama data yang diambil:")
print(df.head())

print(f"\nDistribusi Status Jemuran:\n{df['status_jemuran'].value_counts(normalize=True)}")


5 baris pertama data yang diambil:
   temperature  humidity  rain_value  ldr_value status_jemuran
0          0.0       0.0        1023        216       Tertarik
1          0.0       0.0        1023        216       Tertarik
2          0.0       0.0        1023        216       Tertarik
3          0.0       0.0        1023        216       Tertarik
4          0.0       0.0        1023        216       Tertarik

Distribusi Status Jemuran:
status_jemuran
Tertarik    1.0
Name: proportion, dtype: float64


In [6]:
# Memisahkan fitur (X) dan target (y)
features_for_model = ['temperature', 'humidity', 'rain_value', 'ldr_value']
X = df[features_for_model]
y = df['status_jemuran']

print(f"\nBentuk X (Fitur): {X.shape}")
print(f"Bentuk y (Target): {y.shape}")


Bentuk X (Fitur): (149, 4)
Bentuk y (Target): (149,)


In [7]:
# Mengubah target kategori menjadi angka ('Terjemur' -> 0, 'Tertutup' -> 1, atau sebaliknya)
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
print(f"Mapping Status Jemuran: {list(encoder.classes_)} -> {encoder.transform(encoder.classes_)}")

Mapping Status Jemuran: ['Tertarik'] -> [0]


In [8]:
# Memisahkan data menjadi set pelatihan (training) dan pengujian (testing)
# Gunakan test_size yang wajar, misal 20%
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

print(f"\nBentuk X_train: {X_train.shape}")
print(f"Bentuk X_test: {X_test.shape}")


Bentuk X_train: (119, 4)
Bentuk X_test: (30, 4)


In [10]:
# Scaling fitur numerik
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Pembangunan & Pelatihan Model

In [11]:
model = DecisionTreeClassifier(random_state=42)
print("\nMelatih model Decision Tree Classifier...")
model.fit(X_train_scaled, y_train)
print("Model selesai dilatih!")


Melatih model Decision Tree Classifier...
Model selesai dilatih!


# Evaluasi Model

In [32]:
y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [16]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))
cm = confusion_matrix(y_test, y_pred)


Classification Report:
              precision    recall  f1-score   support

    Tertarik       1.00      1.00      1.00        30

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



c:\Users\USER\OneDrive\Documents\Flask\.venv\Lib\site-packages\sklearn\metrics\_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [ ]:
model_dir = 'Model'
os.makedirs(model_dir, exist_ok=True) # Buat folder jika belum ada

model_filename = os.path.join(model_dir, 'jemuran_ai_model.pkl')
scaler_filename = os.path.join(model_dir, 'jemuran_ai_scaler.pkl')
encoder_filename = os.path.join(model_dir, 'jemuran_ai_encoder.pkl')

joblib.dump(model, model_filename)
joblib.dump(scaler, scaler_filename)
joblib.dump(encoder, encoder_filename)
print(f"Model, Scaler, dan Encoder berhasil disimpan di folder '{model_dir}'.")

Model, Scaler, dan Encoder berhasil disimpan di folder 'Model'.


# Contoh Predict

In [30]:
print("\n--- Contoh Prediksi Menggunakan Data Sensor Terbaru dari Database ---")
try:
    conn_latest = pymysql.connect(**DB_CONFIG)
    query_latest = """
    SELECT 
        temperature, 
        humidity, 
        rain_value, 
        ldr_value
    FROM 
        jemuran_data
    ORDER BY waktu DESC 
    LIMIT 1
    """
    latest_sensor_data = pd.read_sql(query_latest, conn_latest)
    conn_latest.close()

    if not latest_sensor_data.empty:
        print("Data sensor terbaru dari database untuk prediksi:")
        print(latest_sensor_data)

        latest_sensor_data_scaled = scaler.transform(latest_sensor_data[features_for_model])
        prediction_latest_encoded = model.predict(latest_sensor_data_scaled)
        predicted_status_latest = encoder.inverse_transform(prediction_latest_encoded)[0]

        print(f"\nPrediksi Status Jemuran (Terbaru Database): {predicted_status_latest}")
        if predicted_status_latest == 'Tertutup':
            print("Keputusan AI: Jemuran disarankan untuk DITARIK / DITUTUP.")
        else:
            print("Keputusan AI: Jemuran aman untuk TETAP DIJEMUR / DIBUKA.")
    else:
        print("Tidak ada data sensor terbaru di database untuk diprediksi.")

except Exception as e:
    print(f"ERROR: Gagal mengambil data terbaru dari MySQL untuk prediksi: {e}")


--- Contoh Prediksi Menggunakan Data Sensor Terbaru dari Database ---
Data sensor terbaru dari database untuk prediksi:
   temperature  humidity  rain_value  ldr_value
0         28.0      59.0        1023        321

Prediksi Status Jemuran (Terbaru Database): Tertarik
Keputusan AI: Jemuran aman untuk TETAP DIJEMUR / DIBUKA.


C:\Users\USER\AppData\Local\Temp\ipykernel_9688\2224258652.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_sensor_data = pd.read_sql(query_latest, conn_latest)
